In [14]:
from huggingface_hub import InferenceClient
import os
from pathlib import Path


api_key = os.environ["HUGGINGFACE_API_KEY"]
if not api_key:
    raise ValueError("HUGGINGFACE_API_KEY environment variable is not set.")

client = InferenceClient(model="Qwen/Qwen2.5-Coder-32B-Instruct", api_key=api_key)

# 根据传入的文本，调用大模型处理返回结果
def prompt_model(content):
  code = repr(content) 
  prompt = """对以下代码中的 react 组件、函数增加注释，注释使用 jsdoc 格式，注释内容为函数的功能描述，注释位置为函数定义的上一行，注释不会覆盖原代码，不会影响到源代码，
  代码示例：
  // 原代码
  import React from 'react';

  const UserList: React.FC<Props> = ({ list }) => {

    const [users, setUsers] = useState([]);

    // ...
  };

  // 增加注释后的代码
  import React from 'react';
  
  /**
  * UserList 组件，用于渲染用户列表
  * @param {Props} props - 组件的属性对象
  * @param {Array<{ name: string; path?: string }>} props.list - 用户列表数组
  * @returns {JSX.Element | null} - 返回用户列表的 JSX 元素，如果  list 不是数组则返回 null
  */
  const UserList: React.FC<Props> = ({ list }) => {

    const [users, setUsers] = useState([]);

    // ...
  };

  """ + f"""需要修改的代码：{code}
  返回代码字符串，不要带 markdown 标记
  """

  messages = [
      {"role": "system", "content": "你是前端专家，一个乐于助人的小助手."},
      {"role": "user", "content": prompt}
  ]

  try:
    response_content=client.chat_completion(
      messages=messages,
      max_tokens=5000,
      # stream=True,
    )
    response_content = response_content.choices[0].message.content
    print(response_content)
    return response_content
  except Exception as e:
      print(f"An error occurred: {e}")
   


In [1]:
import sys
module_path = '../common/lib'
if module_path not in sys.path:
    sys.path.append(module_path)
import open_dir

open_dir.open_dir_ui()

GridBox(children=(Button(button_style='success', description='选择解析目录', style=ButtonStyle(), tooltip='选择目录，目录下所…

2024-11-16 16:54:22.859 python[30181:682586] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-16 16:54:23.500 python[30181:682586] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
2024-11-16 17:00:39.819 python[30181:682586] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [15]:
from pathlib import Path

text = open_dir.text.value


# 设置起始目录
start_dir = Path(text)

def write_res(content,write_path):
  write_path.write_text(content)

def run(dir_path):
  # 判断是否是目录
  if not dir_path.is_dir():
      print(f"The provided path is not a directory: {start_dir}")
      content = start_dir.read_text()
      print(content)
      exit(1)

  # # 遍历目录
  for file_path in start_dir.rglob("*"):
      print(file_path)
      # 检查文件后缀名
      if file_path.suffix in [".jsx", ".tsx"]:
          # 打印文件路径
          print(f"Found file: {file_path}")
          # 读取文件内容
          try:
              content = file_path.read_text()
              res_content = prompt_model(content)
              # 生成结果写入文件中
              write_path =Path('./data/res.tsx')
              write_res(res_content,file_path)
              print("------")  # 分隔线，用于区分不同文件的内容
          except Exception as e:
              print(f"Error reading file {file_path}: {e}")

run(start_dir)

/Users/liepin/liepin-project/fe-mskh-pc/v6/src/components/business/AgreenChecked/index.tsx
Found file: /Users/liepin/liepin-project/fe-mskh-pc/v6/src/components/business/AgreenChecked/index.tsx
import { getSafetyAgreement } from '@common/js/safetyAgreement';
import { Form, FormInstance, Checkbox } from 'antd';
import React from 'react';

/**
 * AgreenChecked 组件，用于渲染并检查用户是否勾选了增值服务协议的复选框
 * @param {Object} props - 组件的属性对象
 * @param {FormInstance} props.form - antd Form 组件实例，用于表单数据的收集和校验
 * @returns {JSX.Element} - 返回渲染后的表单组件，其中包含一个复选框用于用户接受协议
 */
const AgreenChecked = (props: { form: FormInstance }) => {
  return (
    <Form form={props.form}>
      <Form.Item
        style={{ marginBottom: 0 }}
        name="agreen"
        valuePropName="checked"
        initialValue={true}
        rules={[
          {
            validator: (rule, value) => {
              if (!value) {
                return Promise.reject(new Error('请勾选协议'));
              } else return Promise.resolve();
          